## Prediction rating

In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
df_movie_ratings = pd.read_csv('/home/ignacio/Datasets/Amazon/Data cleaned/ratings_movie_intersect_greater_1.csv')
df_movie_ratings.head()

userId        asin  rating  reviewTime  movieId
0   AGEIT17HENDIS  0764003828     3.0  1185062400     1963
1  A34PAZQ73SL163  0764003828     5.0  1110585600     1963
2  A20IIR0422G3A5  0764003828     5.0  1115078400     1963
3   AW3VZ5O895LRK  0764003828     4.0  1379116800     1963
4  A161ANWB62G3M8  0764003828     5.0  1222041600     1963

In [ ]:
df_books = pd.read_csv('/home/ignacio/Datasets/Amazon/Data cleaned/movie_meta_valid_genres.csv')
df_books.head()

In [ ]:
mean_rating = df_book_ratings['rating'].mean()
mean_rating

## Estimate bi

In [ ]:
df_book_ratings_norm = df_book_ratings.copy()
df_book_ratings_norm['rating'] = df_book_ratings_norm['rating'] - mean_rating
df_book_ratings_norm.head()

In [ ]:
movie_rating_sum = df_book_ratings_norm[['movieId', 'rating']].groupby('movieId').agg('sum')
movie_rating_sum.head()

In [ ]:
movie_rating_count_user = df_book_ratings[['movieId', 'userId']].groupby('movieId').count()
movie_rating_count_user.head()

In [ ]:
movie_rating_count_user['userId'] = movie_rating_count_user['userId'] + 25
movie_rating_count_user.head()

In [ ]:
movie_bi = pd.merge(movie_rating_sum, movie_rating_count_user, how='inner', right_index=True, left_index=True)
movie_bi.head()

In [ ]:
movie_bi['bi'] = movie_bi['rating'] / movie_bi['userId']
movie_bi.head()

In [ ]:
movie_bi = movie_bi.reset_index()
movie_bi.head()

In [ ]:
movie_bi[['movieId', 'bi']].to_csv('/home/ignacio/Datasets/Amazon/Data cleaned/movie_bi.csv', index=False)

## Estimate bu

In [ ]:
df_book_ratings_norm['rating'] = df_book_ratings_norm.apply(
    lambda row : row['rating'] - movie_bi[movie_bi['asin'] == row['asin']]['bi'].values[0], axis = 1)
df_book_ratings_norm.head()

In [ ]:
movie_norm_sum = df_book_ratings_norm[['userId', 'rating']].groupby(['userId']).agg('sum')
movie_norm_sum.head()

In [ ]:
movie_norm_count = df_book_ratings_norm[['userId', 'asin']].groupby(['userId']).count()
movie_norm_count.head()

In [ ]:
movie_norm_count['asin'] = movie_norm_count['asin'] + 10
movie_norm_count.head()

In [ ]:
movie_bu = pd.merge(movie_norm_sum, movie_norm_count, left_index=True, right_index=True)
movie_bu.head()

In [ ]:
movie_bu['bu'] = movie_bu['rating'] / movie_bu['asin']
movie_bu.head()

In [ ]:
movie_bu = movie_bu.reset_index()
movie_bu.head()

In [ ]:
movie_bu[['userId', 'bu']].to_csv('/home/ignacio/Datasets/Amazon/Data cleaned/movie_bu.csv', index=False)

## MAE proof

In [18]:
df_prediction = pd.read_csv('/home/ignacio/Datasets/Amazon/Data cleaned/Recommendation/recommendation_rating_proof.csv')
df_prediction.head()

movieId                    title              genres     director  \
0     2973  Crimes and Misdemeanors  comedy|crime|drama  Woody Allen   
1     1701     Deconstructing Harry        comedy|drama  Woody Allen   
2     1701     Deconstructing Harry        comedy|drama  Woody Allen   
3    32291      Melinda and Melinda        comedy|drama  Woody Allen   
4     5348         Hollywood Ending        comedy|drama  Woody Allen   

   similarity    rating  
0    0.881196  4.750639  
1    0.867732  4.737176  
2    0.867732  4.737176  
3    0.867732  4.737176  
4    0.867732  4.737176

In [19]:
len(df_prediction)

560

In [20]:
df_ratings = df_movie_ratings[df_movie_ratings['userId'] == 'AGEIT17HENDIS']
df_ratings.head()

userId        asin  rating  reviewTime  movieId
0    AGEIT17HENDIS  0764003828     3.0  1185062400     1963
73   AGEIT17HENDIS  0767819764     3.0  1185667200      492
207  AGEIT17HENDIS  0780621654     3.0  1212364800     1701
346  AGEIT17HENDIS  0783226039     4.0  1223596800     2728
401  AGEIT17HENDIS  0783226152     5.0  1282867200     3363

In [22]:
df_ratings_join = pd.merge(df_ratings, df_prediction[['movieId', 'rating']], how='left', left_on='movieId', right_on='movieId')
df_ratings_join.head()

userId        asin  rating_x  reviewTime  movieId  rating_y
0  AGEIT17HENDIS  0764003828       3.0  1185062400     1963  4.706887
1  AGEIT17HENDIS  0767819764       3.0  1185667200      492  4.674300
2  AGEIT17HENDIS  0767819764       3.0  1185667200      492  4.674300
3  AGEIT17HENDIS  0767819764       3.0  1185667200      492  4.584083
4  AGEIT17HENDIS  0767819764       3.0  1185667200      492  4.584083

In [23]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mean_absolute_error(df_ratings_join['rating_x'], df_ratings_join['rating_y'])

0.96643251946304032

In [24]:
mean_squared_error(df_ratings_join['rating_x'], df_ratings_join['rating_y'])

1.4013778339630074